In [2]:
from yuzu.exchanges.binanceus import create_order, get_balances
create_order('sell', 'ALGOUSD')
get_balances()

{'pair': 'ALGOUSD', 'left': 'ALGO', 'right': 'USD', 'left_min': 0.001, 'right_min': 10.0, 'left_precision': 3, 'right_precision': 4, 'curr_price': 0.844}
{'symbol': 'ALGOUSD', 'side': 'SELL', 'type': 'MARKET', 'quantity': 17.862}


{'balances': {'BTC': {'amount': 2e-08, 'value': 0.0009},
  'ETH': {'amount': 9e-06, 'value': 0.0272},
  'USD': {'amount': 15.0579, 'value': 15.0579},
  'LTC': {'amount': 9.1e-07, 'value': 0.0001},
  'XLM': {'amount': 0.0020061, 'value': 0.0006},
  'ALGO': {'amount': 0.01810279, 'value': 0.0153},
  'BUSD': {'amount': 0.00777363, 'value': 0.0078},
  'MATIC': {'amount': 7.4121, 'value': 8.2608}},
 'total': 23.3706}